In [458]:
import onnx
import onnx_graphsurgeon as gs
import onnxruntime as ort
import torch
import numpy as np
from rich import print
from onnx import TensorProto
from onnx import helper,numpy_helper

file_name=r"D:\Learn DL\ONNX\bevdet_r50_ckpt_folded_final_folded.onnx"

# model=onnx.load(file_name)

## Isolation

In [459]:
# session = ort.InferenceSession(model.SerializeToString())
# org_outputs = [x.name for x in session.get_outputs()]
# org_inputs = [x.name for x in session.get_inputs()]

# print(org_outputs,org_inputs

In [460]:
# graph=gs.import_onnx(model)
# tensors=graph.tensors()


# graph.inputs=[tensors['/pts_bbox_head/Concat_35_output_0']]
# graph.outputs=[tensors['/pts_bbox_head/Mul_16_output_0']]

# graph.cleanup()

# sub_model=gs.export_onnx(graph)

# onnx.checker.check_model(sub_model)

# onnx.save(sub_model, "mul_isolated.onnx")

# print("\nONNX model 'mul_isolated.onnx' generated successfully!")

## Add squeeze Unsqueeze

In [509]:
sub_graph=gs.import_onnx(onnx.load("mul_isolated.onnx"))

slice = [node for node in sub_graph.nodes if node.op=="Slice"][0]
less_or_eq = [node for node in sub_graph.nodes if node.op=="LessOrEqual"][0]
great_or_eq = [node for node in sub_graph.nodes if node.op=="GreaterOrEqual"][0]



In [510]:

# Step 2: Create a New Node (Example: Squeeze)
squeeze_output1 = gs.Variable(name="squeeze_output1", dtype=slice.outputs[0].dtype,shape=[500,3])
sq_node1 = gs.Node(op="Squeeze", inputs=[slice.outputs[0]], outputs=[squeeze_output1])

squeeze_output2 = gs.Variable(name="squeeze_output2", dtype=slice.outputs[0].dtype,shape=[500,3])
sq_node2 = gs.Node(op="Squeeze", inputs=[slice.outputs[0]], outputs=[squeeze_output2])

# Step 3: Modify Connections
less_or_eq.inputs[0] = sq_node1.outputs[0]
great_or_eq.inputs[0] = sq_node2.outputs[0]

sub_graph.nodes.extend([sq_node1,sq_node2])  # Add new node to the graph

# Step 4: Save the Modified Model
sub_graph.toposort()


['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 17. 
['\x1b'][E] Function: '_ipython_display_' was not registered for opset 17. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 17. 
['\x1b'][E] Function: '_repr_mimebundle_' was not registered for opset 17. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 17. 
['\x1b'][E] Function: '_repr_html_' was not registered for opset 17. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 17. 
['\x1b'][E] Function: '_repr_markdown_' was not registered for opset 17. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 17. 
['\x1b'][E] Function: '_repr_svg_' was not registered for opset 17. 
['\x1b'][E] Function: '_ipython_canary_method_should_not_exist_' was not registered for opset 17. 
['\x1b'][E] Function: '_repr_png_' was no

Graph main_graph (Opset 17)
Local Functions: []
Inputs: [Variable (/pts_bbox_head/Concat_35_output_0): (shape=[1, 500, 9], dtype=float32)]
Nodes: /pts_bbox_head/Slice_2 (Slice)
	Inputs: [
		Variable (/pts_bbox_head/Concat_35_output_0): (shape=[1, 500, 9], dtype=float32)
		Constant (onnx::Unsqueeze_711): (shape=[1], dtype=int64)
		Constant (/img_view_transformer/Constant_19_output_0): (shape=[1], dtype=int64)
		Constant (/img_view_transformer/Constant_21_output_0): (shape=[1], dtype=int64)
		Constant (/Constant_8_output_0): (shape=[1], dtype=int64)
	]
	Outputs: [
		Variable (/pts_bbox_head/Slice_2_output_0): (shape=[1, 500, 3], dtype=float32)
	]
 (Squeeze)
	Inputs: [
		Variable (/pts_bbox_head/Slice_2_output_0): (shape=[1, 500, 3], dtype=float32)
	]
	Outputs: [
		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
	]
 (Squeeze)
	Inputs: [
		Variable (/pts_bbox_head/Slice_2_output_0): (shape=[1, 500, 3], dtype=float32)
	]
	Outputs: [
		Variable (squeeze_output1): (shape=[500, 3],

In [511]:
nodes=sub_graph.nodes

cast1 = [node for node in sub_graph.nodes if node.name=="/pts_bbox_head/Cast_19"][0]
cast2 = [node for node in sub_graph.nodes if node.name=="/pts_bbox_head/Cast_21"][0]



In [512]:

unsqueeze_output1 = gs.Variable(name="unsqueeze_output1", dtype=np.float32)
unsqueeze_output2 = gs.Variable(name="unsqueeze_output2", dtype=np.float32)
unsqueeze_axes1 = gs.Constant(name="unsqueeze_axes1", values=np.array([0]))  
unsqueeze_axes2 = gs.Constant(name="unsqueeze_axes2", values=np.array([0]))  

unsqueeze_node1 = gs.Node (op="Unsqueeze",inputs=[cast1.outputs[0],unsqueeze_axes1],outputs=[unsqueeze_output1])

unsqueeze_node2 = gs.Node(
    op="Unsqueeze",
    inputs=[cast2.outputs[0],unsqueeze_axes2],
    outputs=[unsqueeze_output2]
)

mul = [node for node in sub_graph.nodes if node.op=="Mul"][0]


mul.inputs[0] = unsqueeze_output1
mul.inputs[1] = unsqueeze_output2

sub_graph.nodes.extend([unsqueeze_node1,unsqueeze_node2])  

reducesum1 = [node for node in sub_graph.nodes if node.op=="ReduceSum"][0]
reducesum2 = [node for node in sub_graph.nodes if node.op=="ReduceSum"][1]
reducesum_axes=gs.Constant(name="reducesum_axes",values=np.array([1]))

# re
reducesum1.inputs[1]=reducesum_axes
reducesum2.inputs[1]=reducesum_axes
sub_graph.toposort()

modified_model = gs.export_onnx(sub_graph)


In [513]:
from onnxsim import simplify

model_simp, check = simplify(modified_model)
onnx.checker.check_model(model_simp)
onnx.save(model_simp, "sq_unsq.onnx")

## Modify Greater than Less Than Node

In [514]:
model=onnx.load("sq_unsq.onnx")

graph=gs.import_onnx(model)

In [515]:
less_or_eq = [node for node in graph.nodes if node.op=="LessOrEqual"][0]
great_or_eq = [node for node in graph.nodes if node.op=="GreaterOrEqual"][0]

less_or_eq,great_or_eq

(/pts_bbox_head/LessOrEqual (LessOrEqual)
 	Inputs: [
 		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
 		Constant (/pts_bbox_head/Constant_144_output_0): (shape=[3], dtype=float32)
 	]
 	Outputs: [
 		Variable (/pts_bbox_head/LessOrEqual_output_0): (shape=[1, 500, 3], dtype=bool)
 	],
 /pts_bbox_head/GreaterOrEqual (GreaterOrEqual)
 	Inputs: [
 		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
 		Constant (/pts_bbox_head/Constant_141_output_0): (shape=[3], dtype=float32)
 	]
 	Outputs: [
 		Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[1, 500, 3], dtype=bool)
 	])

In [516]:


less_or_eq.outputs[0]=gs.Variable(name="/pts_bbox_head/LessOrEqual_output_0",dtype=np.float32,shape=[500,3])
great_or_eq.outputs[0]=gs.Variable(name="/pts_bbox_head/GreaterOrEqual_output_0",dtype=np.float32,shape=[500,3])


tensors =graph.tensors()

less_or_eq_const=[tensors['/pts_bbox_head/Constant_144_output_0']]
great_or_eq_const=[tensors['/pts_bbox_head/Constant_141_output_0']]

less_or_eq.inputs[1]=gs.Constant(name="/pts_bbox_head/Constant_144_output_0",values=np.reshape(less_or_eq_const[0].values,shape=(1,3)))
great_or_eq.inputs[1]=gs.Constant(name="/pts_bbox_head/Constant_141_output_0",values=np.reshape(great_or_eq_const[0].values,shape=(1,3)))



['\x1b'][W] Found distinct tensors that share the same name:
[id: 3048844309456] Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
[id: 3048844667344] Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[1, 500, 3], dtype=bool)
Note: Producer node(s) of first tensor:
[/pts_bbox_head/GreaterOrEqual (GreaterOrEqual)
	Inputs: [
		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
		Constant (/pts_bbox_head/Constant_141_output_0): (shape=[3], dtype=float32)
	]
	Outputs: [
		Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
	]]
Producer node(s) of second tensor:
[]
['\x1b'][W] Found distinct tensors that share the same name:
[id: 3048817269584] Variable (/pts_bbox_head/LessOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
[id: 3048844666832] Variable (/pts_bbox_head/LessOrEqual_output_0): (shape=[1, 500, 3], dtype=bool)
Note: Producer node(s) of first tensor:
[/p

In [517]:
less_or_eq = [node for node in graph.nodes if node.op=="LessOrEqual"][0]
great_or_eq = [node for node in graph.nodes if node.op=="GreaterOrEqual"][0]

less_or_eq,great_or_eq

(/pts_bbox_head/LessOrEqual (LessOrEqual)
 	Inputs: [
 		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
 		Constant (/pts_bbox_head/Constant_144_output_0): (shape=(1, 3), dtype=float32)
 	]
 	Outputs: [
 		Variable (/pts_bbox_head/LessOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
 	],
 /pts_bbox_head/GreaterOrEqual (GreaterOrEqual)
 	Inputs: [
 		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
 		Constant (/pts_bbox_head/Constant_141_output_0): (shape=(1, 3), dtype=float32)
 	]
 	Outputs: [
 		Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
 	])

In [518]:
sub_model=gs.export_onnx(graph)

onnx.checker.check_model(sub_model)

onnx.save(sub_model, "final.onnx")

print("\nONNX model 'final.onnx' generated successfully!")

['\x1b'][W] Found distinct tensors that share the same name:
[id: 3048844309456] Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
[id: 3048844667344] Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[1, 500, 3], dtype=bool)
Note: Producer node(s) of first tensor:
[/pts_bbox_head/GreaterOrEqual (GreaterOrEqual)
	Inputs: [
		Variable (squeeze_output2): (shape=[500, 3], dtype=float32)
		Constant (/pts_bbox_head/Constant_141_output_0): (shape=(1, 3), dtype=float32)
	]
	Outputs: [
		Variable (/pts_bbox_head/GreaterOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
	]]
Producer node(s) of second tensor:
[]
['\x1b'][W] Found distinct tensors that share the same name:
[id: 3048817269584] Variable (/pts_bbox_head/LessOrEqual_output_0): (shape=[500, 3], dtype=<class 'numpy.float32'>)
[id: 3048844666832] Variable (/pts_bbox_head/LessOrEqual_output_0): (shape=[1, 500, 3], dtype=bool)
Note: Producer node(s) of first tensor:


ONNX model 'final.onnx' generated successfully!

## Debugging

In [519]:
model=onnx.load("sq_unsq.onnx")
graph=gs.import_onnx(model)
tensors=graph.tensors()


graph.inputs=[tensors['/pts_bbox_head/Concat_35_output_0']]
graph.outputs=[tensors['squeeze_output2']]

graph.cleanup()

sub_model=gs.export_onnx(graph)

onnx.checker.check_model(sub_model)

session = ort.InferenceSession(sub_model.SerializeToString())

input_value=np.random.randn(1,500,9).astype(np.float32)
inputs = [x.name for x in session.get_inputs()]


output=session.run(None,{inputs[0]:input_value})

onnx.save(sub_model,"debug.onnx")

print(np.array(output).shape)

(1, 500, 3)

In [522]:
import onnxruntime as ort
import onnx
import numpy as np
from collections import OrderedDict

# file="debug.onnx"

file="final.onnx"

ort_session_1 = ort.InferenceSession(file)
org_outputs = [x.name for x in ort_session_1.get_outputs()]

model = onnx.load(file)
for node in model.graph.node:
    for output in node.output:
        if output not in org_outputs:
            model.graph.output.extend([onnx.ValueInfoProto(name=output)])
 
ort_session = ort.InferenceSession(model.SerializeToString())

outputs = [x.name for x in ort_session.get_outputs()]
inputs = [x.name for x in ort_session.get_inputs()]
input_shape = ort_session.get_inputs()[0].shape

input_value=np.random.randn(1,500,9).astype(np.float32)


ort_outs = ort_session.run(None, input_feed={inputs[0]:input_value})


from collections import OrderedDict
ort_outs = OrderedDict(zip(outputs, ort_outs))

for key in ort_outs.keys():
    print(f"Layer : {key} Shape :{ort_outs[key].shape}")

Layer : /pts_bbox_head/Mul_16_output_0 Shape :(1, 500)

Layer : /pts_bbox_head/Slice_2_output_0 Shape :(1, 500, 3)

Layer : squeeze_output2 Shape :(500, 3)

Layer : /pts_bbox_head/GreaterOrEqual_output_0 Shape :(500, 3)

Layer : /pts_bbox_head/LessOrEqual_output_0 Shape :(500, 3)

...

Layer : /pts_bbox_head/Not_3_output_0 Shape :(500,)

Layer : /pts_bbox_head/Cast_19_output_0 Shape :(500,)

Layer : /pts_bbox_head/Cast_21_output_0 Shape :(500,)

Layer : unsqueeze_output1 Shape :(1, 500)

Layer : unsqueeze_output2 Shape :(1, 500)

## MSE

In [524]:

session = ort.InferenceSession("final.onnx")
input_value=np.random.randn(1,500,9).astype(np.float32)
output1= session.run(None, input_feed={inputs[0]:input_value})


session = ort.InferenceSession("mul_isolated.onnx")
input_value=np.random.randn(1,500,9).astype(np.float32)
output2= session.run(None, input_feed={inputs[0]:input_value})



original_output = torch.tensor(output1[0], dtype=torch.float32)
modified_output = torch.tensor(output2[0],dtype=torch.float32)


mse = torch.nn.functional.mse_loss(original_output, modified_output)

print(mse)

tensor(0.)